# Ayudantía 04: Iterables y Generadores 🔁

## Ayudantes 👾

Y sus recomendaciones semanales 🎵

- S1: Enzo Acosta
  - [Still Beating - Mac DeMarco](https://www.youtube.com/watch?v=Z1zPvvh3KxU)
- S2: Bastián Pérez
  - [Rivers Is a Vampire - Bear Ghost](https://www.youtube.com/watch?v=g-vpodj0Tqk)
- S3: Clemente Campos
  - [promise - key vs. locket](https://www.youtube.com/watch?v=hxMG9AELpKE)
- S4: Carlos Olguín
  - [TOOTIMETOOTIMETOOTIME - The 1975](https://www.youtube.com/watch?v=4fxPQUKfim4)
- S5: Carlos Martel
  - [Chlorine - Twenty One Pilots](https://www.youtube.com/watch?v=eJnQBXmZ7Ek)

## Contenidos 📖

- Iterables
- Generadores
- Listas Ligadas

## DCChilean Express 🚂

El magnífico tren DCChilean Express está pronto a partir rumbo a Castletown!

## Ejercicio 1: ¡Prepárense para abordar! 🚉

Antes de partir, te encomiendan una tarea muy importante. Debes revisar la carga y ver cuáles productos están más próximos a vencer. El problema es que la mercadería no está ordenada y esto dificulta tu trabajo considerablemente. Te sientas a pensar por un momento, y por el cansancio te quedas dormido. Tienes un sueño muy extraño, donde imaginas toda la mercadería como un objeto en Python. Si tan solo pudieras meter este objeto a un ciclo `for` y que te fuera entregando los productos ordenados por fecha de vencimiento... en este momento, despiertas y tienes totalmente claro lo que tienes que hacer: un **iterable**!!. 

Están definidas dos clases que representan una **mercadería** y una **fecha**. También está definida la función `fecha_sort`, que te ayudará más adelante. Deberás completar las clases `ConjuntoMercaderia` e `IteradorConjuntoMercaderia`, de modo de convertir a la primera en un **iterable**. Debes tener lo siguiente en cuenta:

- Al iterar sobre una instancia de `ConjuntoMercaderia`, se deben ir entregando las mercaderías por orden de fecha de vencimiento, de más próxima a más lejana.
- Las instancias de `ConjuntoMercaderia` deben ser indexables de la misma manera, es decir, `conjunto_mercaderia[i]` debe entregar la i-ésima mercadería más próxima a vencer.

Tip: la función `sorted` te entrega una lista ordenada de menor a mayor. Recuerda cómo funciona el parámetro `key` de ésta.



In [ ]:
from __future__ import annotations
from utils import Fecha, Mercaderia

def fecha_sort(carga: Mercaderia):
    return (carga.fecha_vencimiento.ano, carga.fecha_vencimiento.mes, carga.fecha_vencimiento.dia)

class ConjuntoMercaderia:

    def __init__(self, nombre: str, sort_key: function):
        self.nombre = nombre
        self.cargas = []
        self.sort_key = sort_key
    
    def __iter__(self):
        return IteradorConjuntoMercaderia(self.cargas.copy(), self.sort_key)
    
    def cargar(self, ruta: str):
        with open(ruta, encoding="UTF-8") as archivo:
            for linea in archivo:
                datos = linea.strip().split(",")
                fecha = Fecha(*(int(i) for i in datos[1].split(";")))
                carga = Mercaderia(datos[0], fecha, int(datos[2]), datos[3])
                self.cargas.append(carga)

    def __getitem__(self, index: int):
        if index >= len(self.cargas):
            raise IndexError
        ordenada = sorted(self.cargas, key=self.sort_key)
        return ordenada[index]

class IteradorConjuntoMercaderia:

    def __init__(self, cargas: list, sort_key: function):
        self.cargas = cargas
        self.sort_key = sort_key
    
    def __next__(self):

        if self.cargas == []:
            raise StopIteration
        
        ordenada = sorted(self.cargas, key=self.sort_key)
        i = self.cargas.index(ordenada[0])
        return self.cargas.pop(i)


mercaderias = ConjuntoMercaderia("DCChilean Express", fecha_sort)
mercaderias.cargar("mercaderia.csv")

for carga in mercaderias:
    print(f"{carga.cantidad} de {carga.nombre} vencen el {carga.fecha_vencimiento.dia}/{carga.fecha_vencimiento.mes}/{carga.fecha_vencimiento.ano}")

print("-" * 40)
print(f"El séptimo producto en vencer es: {mercaderias[6].cantidad} de {mercaderias[6].nombre} vencen el {mercaderias[6].fecha_vencimiento.dia}/{mercaderias[6].fecha_vencimiento.mes}/{mercaderias[6].fecha_vencimiento.ano}")
print(f"El último producto en vencer es: {mercaderias[-1].cantidad} de {mercaderias[-1].nombre} vencen el {mercaderias[-1].fecha_vencimiento.dia}/{mercaderias[-1].fecha_vencimiento.mes}/{mercaderias[-1].fecha_vencimiento.ano}")


## Ejercicio 2: Próxima estación... 🛤️

Ahora que saben con qué cargar el DCChilean Express es momento de asignar la mercadería a los distintos vagones. ¡Pero ojo! El tren pasará por varias estaciones, y para que la descarga de la mercadería sea eficiente, se deben ordenar los vagones según el orden en que se desacoplarán del tren. ~~Para simplificar~~ Por suerte, las estaciones están ordenadas alfabéticamente en la línea de ferrocarril. Por lo que deberás ordenar los vagones de la siguiente forma:

> Los primeros vagones del tren deben corresponder a las estaciones que se entregarán al final del recorrido, mientras que los últimos vagones estarán destinados a las estaciones del inicio del recorrido. El recorrido por las estaciones se hace en orden alfabético. En caso de que haya varios vagones con destino a la misma estación, se deben ordenar según la fecha de vencimiento de su carga: primero se colocan los vagones con mercadería que vence más tarde y, detrás de ellos, los que tienen una fecha de vencimiento más próxima.

Para que el tren pueda lograr su función, este debe permitir que se agreguen y se desacoplen vagones de él. Para esto, deberás completar los siguientes métodos de la clase `Tren`:

- `agregar_vagon`: este método debe crear un nuevo objeto `Vagon` y ubicarlo en el tren según el orden definido arriba. La clase `Vagon` tiene el método `__gt__` implementado, por lo que puedes comparar dos instancias con el operador `>` según el orden indicado arriba. Para la inserción hay tres opciones:
    - Si la lista está vacía, el nuevo se convierte en cabeza y cola.
    - Si el nuevo es mayor que la cabeza, se inserta al inicio y pasa a ser la nueva cabeza.
    - En cualquier otro caso, hay que recorrer los vagones del tren hasta encontrar la posición correcta. Al final siempre se incrementa el largo del tren.

- `desacoplar_vagon`: este método quita y devuelve el último vagón del tren, decrementando el largo de éste. De nuevo, hay tres opciones:
    - Si el tren está vacío, retorna ``None``.
    - Si tiene un solo elemento, lo retorna y cambia ``self.cabeza`` y ``self.cola`` a ``None``.
    - Si hay más de un vagón en el tren, se entrega el último, teniendo cuidado de actualizar el valor de la nueva cola y eliminando el atributo `siguiente` de esta última.

In [ ]:
class Vagon:

    def __init__(self, mercaderia: Mercaderia):
        self.carga = mercaderia
        self.siguiente: Vagon | None = None  # Puede ser Vagon o None
    
    def __str__(self) -> str:
        return f'[{self.carga}]->'
    
    def __gt__(self, otro_vagon: Vagon) -> bool:
        '''
        Definimos el operador > para comparar vagones. Esto encapsula la
        lógica de orden dentro de la clase Vagon, evitando
        duplicar código de comparación en otras partes del programa.
        '''

        if not isinstance(otro_vagon, Vagon):
            raise TypeError(f'Argumento de tipo {type(otro_vagon)},'
                            f' cuando se esperaba un {type(self)}.')
        
        if (self.carga.estacion == otro_vagon.carga.estacion):
            return self.carga.fecha_vencimiento > otro_vagon.carga.fecha_vencimiento
        return self.carga.estacion > otro_vagon.carga.estacion
    

class Tren:

    def __init__(self):
        self.cabeza = None
        self.cola = None
        self.largo = 0

    def agregar_vagon(self, mercaderia: Mercaderia) -> None:
        nuevo_vagon = Vagon(mercaderia)
        anterior = None
        actual = self.cabeza

        if actual is None:
            self.cabeza = nuevo_vagon
            self.cola = nuevo_vagon
        elif nuevo_vagon > actual:
            nuevo_vagon.siguiente = actual
            self.cabeza = nuevo_vagon
        else:
            insertado = False

            while not insertado:
                anterior = actual
                actual = actual.siguiente

                if actual is None:
                    anterior.siguiente = nuevo_vagon
                    self.cola = nuevo_vagon
                    insertado = True
                elif nuevo_vagon > actual:
                    nuevo_vagon.siguiente = actual
                    anterior.siguiente = nuevo_vagon
                    insertado = True
        self.largo += 1
    
    def desacoplar_vagon(self) -> Vagon | None:
        self.largo -= 1
        actual = self.cabeza

        if actual is None:
            return None
        elif actual.siguiente is None:
            self.cabeza = None
            self.cola = None
            return actual
        else:
            while actual.siguiente != self.cola:
                actual = actual.siguiente

            cola_anterior = actual.siguiente
            actual.siguiente = None
            self.cola = actual

            return cola_anterior
        
    def __str__(self) -> str:
        if self.largo == 0:
            return 'Tren sin carga'
        
        tren = ''
        actual = self.cabeza
        while actual is not None:
            tren += str(actual)
            actual = actual.siguiente
        return tren


tren = Tren()

with open('mercaderia.csv', encoding='utf-8') as archivo:
    print('Acoplando vagones!')
    for linea in archivo:
        datos = linea.strip().split(",")
        fecha = Fecha(*(int(i) for i in datos[1].split(";")))
        mercaderia = Mercaderia(datos[0], fecha, int(datos[2]), datos[3])
        tren.agregar_vagon(mercaderia)
        print(tren)
        
print(f'Tren, de largo {tren.largo}, listo para comenzar el viaje!')
while tren.cabeza is not None:
    tren.desacoplar_vagon()
    print(tren)

print('Tren llegó al destino entregando exitosamente toda su carga.')


## Ejercicio 3: ¿Todo en orden? 

Gracias a ti y a tus habilidades de programación, el magnífico DCChilean Express llegó a Castletown, distribuyendo eficientemente bienes a lo largo del país. Ahora, en la estación terminal, te solicitan el historial de descarga para compararlo con el manifiesto de carga y corroborar que todo salió en orden. Lamentablemente, su sistema está caído, ¡oh no! Cuando todo parece perdido, recuerdas tus habilidades y les ofreces tu ayuda, para así salvar el día.

Para lograrlo, deberás completar la función **verificador_mercaderias**, que recibe un `IteradorConjuntoMercaderia` ordenado con la función `estacion_fecha_sort` y un generador con instancias de mercaderías en el mismo orden. Esta función debe seguir el siguiente comportamiento:
- Imprimir el mensaje `Comenzando verificación de mercaderías...`.
- Recorrer ambos iteradores en paralelo utilizando el método `__next__` de cada uno y comprobar que todos los pares de elementos coincidan. Si en algún momento se encuentra una diferencia, se debe mostrar el mensaje `Error detectado! Mercaderías difieren!`. En caso de que todos los elementos sean iguales, se imprime `Mercaderías verificadas, todo en orden!`.

Puedes asumir que ambos iteradores tendrán el mismo número de elementos.

Ojo: no debes preocuparte de que ninguno de los generadores se acabe, gracias al bloque `try`/`except`. La semana que viene aprenderás cómo funciona esto 👀.

In [ ]:
from typing import Generator
from utils import obtener_mercaderias

def estacion_fecha_sort(carga: Mercaderia):
    return (carga.estacion, carga.fecha_vencimiento)

def verificador_mercaderias(iter_historial: IteradorConjuntoMercaderia, gen_mercaderias: Generator):
    try:
        print('Comenzando verificación de mercaderías...')
        sig_historial = next(iter_historial)
        sig_mercaderias = next(gen_mercaderias)
        while True:
            print(f'¿{sig_historial} == {sig_mercaderias}?')
            if sig_historial.nombre != sig_mercaderias.nombre \
               or sig_historial.fecha_vencimiento != sig_mercaderias.fecha_vencimiento \
               or sig_historial.cantidad != sig_mercaderias.cantidad \
               or sig_historial.estacion != sig_mercaderias.estacion:
                print('Error detectado! Mercaderías difieren!')
                break
            sig_historial = next(iter_historial)
            sig_mercaderias = next(gen_mercaderias)

    except StopIteration:
        print('Mercaderías verificadas, todo en orden!')

historial_descarga = ConjuntoMercaderia('DCChilean Express', estacion_fecha_sort)
historial_descarga.cargar('mercaderia.csv')
manifiesto_carga = obtener_mercaderias('mercaderia.csv', estacion_fecha_sort)

# LLama a la función de verificación
verificador_mercaderias(iter(historial_descarga), manifiesto_carga)